![Département de Mathématiques](https://ktzanev.github.io/logolabopp/ul-fst-math/ul-fst-math_100.gif)

# TP 2 - Les équations différentielles autonomes de dimension 2

Dans ce deuxième TP nous allons étudier numériquement des équations autonomes du premier ordre de dimension 2.<br>
Il s'agit d'exemples vus dans les feuilles de 4 à 6 de TD.

**Attention :** Pour que le code entre les exercices soit au maximum similaire et permettre de faire des copier/coller, nous utilisons plusieurs fois les mêmes noms pour des fonctions différentes. **Ceci est une très mauvaise pratique qui mène à des résultats imprévisibles si on n'exécute pas les cellules dans l'ordre.**

On commence par charger les bibliothèques « standards » dont on aura besoin.

In [ ]:
# numpy pour les calculs (vectoriels)
import numpy as np
# matplotlib (référencé comme `plt` ici) pour les graphiques
from matplotlib import pyplot as plt
# la bibliothèque pour tracer les trajectoires
from scipy.integrate import odeint
# la bibliothèque qui permet d'utiliser (le décorateur) @interact
from ipywidgets import interact

## Quelques fonctions utiles

### Le champ de vecteurs

De façon similaire à la feuille de TP1, nous allons créer une fonction `champ_vectoriel`, mais avec les différences suivantes :
- la fonction `F` passé en paramètre est la fonction correspondante à l'équation autonome de Cauchy $Y'=F(Y)$.
- elle a un paramètre supplémentaire `normalise` qui indique s'il faut normaliser ou pas le champ de vecteurs.

La fonction `F` passé en paramètre doit opérer sur un **vecteur** `Y` (de dimension $2\times n$) qui représente les $n$ points de la grille où on affiche le champ de vecteurs.

In [ ]:
# affiche le champ de vecteurs sur la figure courante
# - F : représente la fonction de Y'=F(Y)
# - xmin, Xmax, ymin, ymax : détermine la fenêtre d'affichage
# - normalise : si `False` le champ n'est pas normalisé
# - step : est le pas de discrétisations en chaque direction
def champ_vectoriel(F, xmin, xmax, ymin, ymax, normalise=True, step=.5, **kw):
    X = np.arange(xmin, xmax, step)  # abscisses des points de la grille
    Y = np.arange(ymin, ymax, step)  # ordonnées des points de la grille
    U, V = F(np.meshgrid(X, Y))  # les composantes du champ de vecteurs
    if normalise:
        M = np.hypot(U, V)  # calcule la norme du vecteur (U,V)
        M[M == 0] = 1  # évite la division par 0
        U /= M  # normalise la composante U
        V /= M  # normalise la composante V
    return plt.quiver(X, Y, U, V, angles='xy', **kw)

### Les courbes de niveau

La fonction `lignes_niveaux` permet de tracer les courbes de niveaux de la fonction `h` passée en premier paramètre.

In [ ]:
# la fonction qui trace les lignes de niveaux de h qui passe par les points xy
# - h : une fonction de R^2 dans R
# - xmin, xmax, ymin, ymax : le limites du tracé
# - xy : une liste de points de la forme [[x1,x2,...],[y1,y2,...]] par lesquelles passent les niveaux
# - N : le nombre de points de la discrétisation de x et de y
# - fmt : le format de la valeur de la courbe de niveau
# - alpha : le niveau de transparence entre 0 (transparent) et 1 (opaque)
def lignes_niveaux(h, xmin, xmax, ymin, ymax, xy, N=100, fmt='%.1f', alpha=.5):
    C = np.sort(h(np.array(xy)))
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = h(np.meshgrid(x, y))
    lignes_niveau = plt.contour(x, y, z, levels=C, alpha=alpha)
    plt.clabel(lignes_niveau, C, fmt=fmt)

### Ajustements de F

#### Rajout du paramètre temps

Comme la fonction `odeint` exige deux paramètres, un vecteur et un paramètre temps `t`, on doit rajouter un `t` à la fonction `F` correspondant à l'équation autonome $Y'=F(Y)$. La fonction utilitaire `avec_t` fait ça : on lui passe une fonction à un paramètre `Y` et elle retourne une fonction à deux paramètres : `Y` et `t`.

In [ ]:
avec_t = lambda F: lambda Y, t: F(Y)

### Ralentir le temps en dehors de la fenêtre d'affichage

Si nous voulons afficher les trajectoires dans une fenêtre à priori nous ne savons pas dans quel intervalle il faut faire varier le temps. Si on donne un temps trop court, on n'affiche qu'un bout de l'orbite, si on donne un temps trop long la méthode `odeint` peut « exploser » (soit pour des raisons « théoriques » car il y a une explosion en temps fini, soit pour des raisons « numériques »).

Pour éviter cela nous pouvons modifier `F` en dehors de la fenêtre d'affichage pour éviter ces problèmes, tout en gardant les même orbites dans la fenêtre d'affichage (où `F` reste non modifiée). 

C'est ce qu'on fait avec la fonction `ralentir(F, xmin, xmax, ymin, ymax)` qui retourne une version « ralentie » de `F` en dehors de la fenêtre `[xmin, xmax, ymin, ymax]`.

In [ ]:
# C'est une fonction C^1 qui :
#  - est égale à 1 dans le disque de rayon r
#  - décroit exponentiellement en dehors de ce disque
def cutoff(Y, r):
    nY = np.maximum(Y[0] * Y[0] + Y[1] * Y[1] - r * r, 0)
    return np.exp(-nY)

# la fonction qui « ralentie » F en dehors de la fenêtre [xmin, xmax, ymin, ymax]
def ralentir(F, xmin, xmax, ymin, ymax):
    r = max(abs(xmax), abs(xmin), abs(ymax), abs(ymin)) * np.sqrt(2)
    return lambda Y: F(Y) * cutoff(Y, r)

## Exemple 1 : TD5 - Exercice 2

### 1) La fonction de Cauchy

La fonction dans cet exercice ne dépend pas de paramètres :
$$ 
    F(x,y) = (x^2-y, x)
$$

nous définissons cette fonction $F(Y)$ pour $Y=(x,y)$.

In [ ]:
F = lambda Y: np.array([Y[0] * Y[0] - Y[1], Y[0]])

# vérification
assert np.all(F(np.array([[0, 1], [0, 2]])) == np.array([[0, -1], [0, 1]]))  # F(0,0)=(0,0) et F(1,2)=(-1,1)

### 2) Le graphique « dynamique » avec les solutions approchées 

Ici, comme en TP1, nous allons afficher le champ de vecteurs et les solutions approchées obtenues par `odeint`. Comme ces solutions « explosent » numériquement dans certaines conditions, nous allons utiliser par défaut une version « ralentie » de `F`.

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -4, 4, -1.5, 2

# choix des paramètres et de la condition initiale grâce à des curseurs
@interact(x_0=(xmin, xmax, .1), y_0=(ymin, ymax, .1), normalise=True, ralentie=True)
# le dessin pour la condition initiale (x,y)
def dyn_solution(x_0=0, y_0=0, normalise=True, ralentie=True):

    # taille (maximale) de la figure en pouces (ici 14" x 7")
    plt.figure(figsize=(14, 7))
    # et on préserve les proportions des axes
    plt.axis('scaled')
    # on détermine la fenêtre d'affichage
    plt.axis([xmin, xmax, ymin, ymax])

    # on trace les isoclines (qui contiennent les axes)
    plt.plot([0, 0], [ymin, ymax], "m")
    plt.plot([xmin, xmax], [0, 0], "r")
    # l'isocline infinie
    u = np.linspace(xmin, xmax, 100)
    plt.plot(u, u * u, "r")

    # on dessine le champ normalisé
    champ_vectoriel(F, xmin, xmax, ymin, ymax, normalise=normalise, width=.001)

    # une solution particulière
    u = np.linspace(2 * xmin, 2 * xmax, 100)
    plt.plot(u / 2, u * u / 4 - 1 / 2, "g")

    # ralentir ou pas
    Fr = ralentir(F, xmin, xmax, ymin, ymax) if ralentie else F
    # la fonction F avec le paramètre t en plus pour `odeint`
    Ft = avec_t(Fr)
    # le temps
    t = np.linspace(0, 5, 100)
    # la partie de la courbe pour t dans [0,3] (le futur)
    x, y = odeint(Ft, [x_0, y_0], t).transpose()
    plt.plot(x, y, "b")
    # la partie de la courbe pour t dans [-3,0] (le passé)
    x, y = odeint(Ft, [x_0, y_0], -t).transpose()
    plt.plot(x, y, "g--")

    # le point de départ (la condition initiale)
    plt.plot(x_0, y_0, 'or')
    # le titre
    plt.title(f"La solution qui vérifie x(0)={x_0:4.1f}, y(0)={y_0:4.1f}")
    # et on affiche tout ça
    plt.show()

## Exemple 2 : TD4 - Exercice 1 et TD6 - Exercice 3

### 1) La fonction de l'équation de Cauchy

Dans la deuxième partie de l'exercice 1 de la feuille 4 nous avons étudié un système de deux équations deux inconnues dépend de deux paramètres $a > 0$ et $r > 0$ qui se réduit au problème de Cauchy autonome :
$$
    F_{r,a}(Y,t) = -rSI,rSI-aI \text{ pour } Y=(S,I).
$$
L'exercice 3 de la feuille 6 étudie le même système avec $a=1$ et $r=2$, mais il rajoute l'étude de une intégrale première et d'une fonction de Liapounov.

Pour cette raison nous définissons la fonction `FF(r, a)` qui retourne une fonction `F(Y)` qui opère sur le 2-vecteur `Y`. Ainsi `Y[0]` représente la quantité $S$ et `Y[1]` représente la quantité $I$.

*Remarque : Comme il s'agit d'une équation autonome ici, le paramètre `t` n'est pas utilisé pour le calcul de `F`.*

In [ ]:
# FF(a,b) retourne la fonction F
# On peut le faire avec `lambda` ainsi :
# FF = lambda r, a: lambda Y: np.array([-r * Y[0] * Y[1], r * Y[0] * Y[1] - a * Y[1]])
# Mais on peut le faire aussi avec `def` de façon plus « éloquente »
def FF(r, a):
    def F(Y):
        S, I = Y  # S = Y[0], I = Y[1]
        Fx = -r * S * I
        Fy = r * S * I - a * I
        return np.array([Fx, Fy])

    return F

# test
assert np.all(FF(1, 2)([3, 4]) == np.array([-12, 4])), "Pour r=1 et a=2, la fonction F vaut [-12, 4] en [3, 4]."

### 2) Une intégrale première

Une intégrale première que nous avons trouvé en TD pour $S>0$ (et $I>0$) est
$$
    h_{a,r}(S,I) = I + S - \frac{a}{r}\log(S).
$$

In [ ]:
# la fonction hh construit et retourne la fonction h sur la base des paramètres a,b,c,d.
def hh(r, a):
    def h(Y):
        S, I = Y
        return I + S - np.log(S) * a / r

    return h

Pour représenter les courbes de niveau de la fonction `h = hh(a,r)` nous allons utiliser la fonction définie plus haut `lignes_niveaux` comme suit (avec $a=1$ et $r=2$, comme dans l'exercice 3 de la feuille de TD °6).

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -.1, 3, -.1, 3

# choix des paramètres du problème
r = 2
a = 1

# taille (maximale) de la figure en pouces (ici 14" x 7")
plt.figure(figsize=(14, 7))
# et on préserve les proportions des axes
plt.axis('scaled')
# on détermine la fenêtre d'affichage
plt.axis([xmin, xmax, ymin, ymax])
# on trace les axes (qui coïncident avec l'isocline infinie)
plt.plot([xmin, xmax], [0, 0], "r")  # l'abscisse = points fixes
plt.plot([0, 0], [ymin, ymax], "r")
# si r est 0 ni l'isocline I_0, ni notre intégrale première, sont définies
if r != 0:
    # l'isocline I_0 (sans les points fixes = abscisse)
    plt.plot([a / r, a / r], [ymin, ymax], "m")
    # les courbes de niveaux
    h = hh(r, a)
    e = 1e-5
    yl = np.linspace(e, ymax, 5)[1:]
    xl = np.ones_like(yl) * a / r
    lignes_niveaux(h, e, xmax, e, ymax, [xl, yl])

# on dessine le champ normalisé
F = FF(r, a)
champ_vectoriel(F, xmin, xmax, ymin, ymax, width=.0005)

# le titre
plt.title(f"Le champ de vecteur et intégrale première pour a={a}, r={r}")
# et on affiche tout ça
plt.show()

### 3) Une fonction de Liapounov

La fonction de Liapounov que nous avons vu en TD (pour $a=1$ et $r=2$, mais en réalité elle ne dépend pas de ces paramètres) est la fonction :
$$
    l(S, I) = S + I.
$$

In [ ]:
liapunov = lambda Y: Y[0] + Y[1]

Pour représenter les courbes de niveaux de cette fonction de Liapounov on peut procéder comme avec l'intégrale première.

### 4) Le graphique « dynamique » avec les solutions approchées

Nous somme prêt maintenant d'intégrer tout ça dans une image « dynamique » qui contient :
- le champ de vecteurs (normalisé ou pas);
- les isoclines $0$ et $\infty$ que nous avons trouvé en TD ;
- le point de départ $(t_0, y_0)$ ;
- l'orbite de la solution approchée obtenue avec `odeint` qui vérifie $y(t_0) = y_0$.

**Remarque :** Dans cet exemple nous n'avons pas d'explosion numérique (les orbites sont bornées), donc nous n'avons pas besoin de « ralentir » `F`.

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -.1, 10, -.1, 3

# options possibles
options_champ = {"ne pas afficher": None, "normalisé": True, "non normalisé": False}
options_niveaux = {"ne pas afficher": None, "fonction de Liapounov": 1, "intégrale première": 2}

# choix des paramètres et de la condition initiale grâce à des curseurs
@interact(r=(1, 10), a=(1, 10), x_0=(0, xmax, .5), y_0=(0, ymax, .5), champ=options_champ, niveaux=options_niveaux)
# le dessin pour la condition initiale (x_0,y_0)
def dyn_solution(r=1, a=3, x_0=4, y_0=2, champ=True, niveaux=2):

    # taille (maximale) de la figure en pouces (ici 14" x 7")
    plt.figure(figsize=(14, 7))
    # et on préserve les proportions des axes
    plt.axis('scaled')
    # on détermine la fenêtre d'affichage
    plt.axis([xmin, xmax, ymin, ymax])

    # on trace les axes (qui coïncident avec l'isocline infinie)
    plt.plot([xmin, xmax], [0, 0], "r")  # l'abscisse = points fixes
    plt.plot([0, 0], [ymin, ymax], "r")
    # l'isocline I_0 (sans les points fixes = abscisse)
    plt.plot([a / r, a / r], [ymin, ymax], "m")

    # les courbes de niveaux
    if niveaux is not None:
        # soit de l'inégale première, soit de Liapounov
        g = liapunov if niveaux == 1 else hh(r, a)

        # on dessine les niveaux qui passe par [a/r,1], [a/r,2],...
        yl = np.linspace(1, ymax, 5)
        xl = np.ones_like(yl) * a / r
        # pour éviter log(0) dans l'intégrale première nous commençons S à 10^-5
        lignes_niveaux(g, 1e-5, xmax, 0, ymax, [xl, yl])

    F = FF(r, a)
    # on dessine le champ normalisé
    if champ is not None:
        champ_vectoriel(F, xmin, xmax, ymin, ymax, normalise=champ, width=.0005)

    # la fonction F avec le paramètre t en plus pour `odeint`
    Ft = avec_t(F)
    # le temps
    t = np.linspace(0, 3, 100)
    # la partie de la courbe pour t dans [0,3] (le futur)
    x, y = odeint(Ft, [x_0, y_0], t).transpose()
    plt.plot(x, y, "b")
    # la partie de la courbe pour t dans [-3,0] (le passé)
    x, y = odeint(Ft, [x_0, y_0], -t).transpose()
    plt.plot(x, y, "g--")

    # le point de départ (la condition initiale)
    plt.plot(x_0, y_0, 'or')
    # le titre
    plt.title(f"Pour a={a}, r={r}, la solution qui vérifie avec S(0)={x_0:4.1f}, I(0)={y_0:4.1f}")
    # et on affiche tout ça
    plt.show()

## Feuille de TD4, Exercice 2

Dans cet exercice nous avons étudié le système différentiel dit de Lotka-Volterra, où a, b, c, d paramètres
(strictement) positifs :
$$
  \begin{align}
    \dot{x} &= \phantom{-}ax - bxy\\
    \dot{y} &= -cy + dxy
  \end{align}
$$

### 1) La fonction de Cauchy

Définissez une fonction `FF(a, b, c, d)` qui en fonction des paramètres `a,b,c,d` retourne la fonction `F` correspondante à l'équation autonome de Cauchy $Y'=F(Y)$ avec $Y=(x,y)$.

In [ ]:
# La fonction FF(a,b,c,d) retourne la fonction F qui prend un 2-vecteur comme paramètre

# test
F = FF(1, 2, 3, 4)  # la fonction F en fcontion des paramètres a=1, b=2, c=3, d=4
assert np.all(F([[0, 1], [0, 2]]) == np.array([[0, -3], [0, 2]])), "Il faut avoir F(0,0)=(0,0) et F(1,2)=(-3,2) pour ces paramètres"

### 2) Intégrale première

Définissez une fonction `hh(a, b, c, d)` qui en fonction des paramètres `a,b,c,d` retourne la fonction `h` correspondante à l'intégrale première trouvée en TD 
$$
    h(x,y) = -a\log(y)-by+c\log(x)-dx.
$$

In [ ]:
# La fonction hh(a,b,c,d) retourne la fonction h qui prend un 2-vecteur Y=(x,y) comme paramètres

# test
h = hh(0, 1, 2, 3)  # la fonction h en pour les paramètres a=0, b=1, c=2, d=3
Y = np.ones((2, 3))  # Y = [[1,1,1],[1,1,1]]
x, y = Y
assert np.all(h(Y) == -x - 3 * y), "Il faut avoir F(1,1)=-4 pour ces paramètres"
Y = np.array([2, 3])
assert f"{hh(1, 2, 3, 4)(Y):.2f}" == "-10.82", f"{hh(1, 2, 3, 4)(Y):.2f} == -10.82 ?"

### 3) Le graphique « dynamique » avec les solutions approchées

Inspirer vous des exemples traités pour afficher un graphique dynamique qui : 
- permet de choisir les paramètres `a,b,c,d` entre 0 et 10 ;
- permet de choisir la condition initiale `x_0, y_0` entre 0 et 10 ;
- qui (qaund c'est possible) affiche plusieurs courbes de niveaux ;
- qui (qaund c'est possible) les isoclines $I_0$ et $I_\infty$ ;
- qui affiche le champ de vecteurs (normalisé)
- contient la solution approchée obtenue avec `odeint`

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -1, 10, -1, 10

# choix des paramètres et de la condition initiale grâce à des curseurs
@interact(a=(0, 10), b=(0, 10), c=(0, 10), d=(0, 10), x_0=(0, xmax, .5), y_0=(0, xmax, .5))
# le dessin avec la solution pour la condition initiale (x_0,y_0)
def dyn_solution(a=7, b=2, c=7, d=2, x_0=3.5, y_0=3.5):


### 4) Observations

1. Que ce passe t-il quand $a=0$ ? Reconnaissez-vous un système étudié ?
1. Et quand $a=0$ et $c=0$ qu'observe t-on ? Comment on peut expliquer ça avec la signification « biologique » des paramètres ? 

**Solution :**


## Feuille de TD5, Exercice 1

### 1) La fonction de Cauchy

La fonction du problème de Cauchy étudié dans cet exercice ne dépend pas de paramètres :
$$ 
    F(Y) = \big(x(1+\frac{3}{x^2+y^2+2}),-y(1-\frac{3}{x^2+y^2+2})\big) \text{ pour } Y=(x,y).
$$

Définissez cette fonction.

In [ ]:
# Définition de la fonction F

# test
assert np.all(F([[0, 1], [1, 0]]) == np.array([[0, 2], [0, 0]])), "F(0,0) = (0,0) et F(1,1) = (0,0)"

### 2) Le graphique « dynamique » avec les solutions approchées

Inspirer vous des exemples traités pour afficher un graphique dynamique qui : 
- permet de choisir la condition initiale `x_0, y_0` entre 0 et 10 ;
- afiche les isoclines $I_0$ et $I_\infty$ (trouvées en TD);
- qui affiche le champ de vecteurs (normalisé ou pas en fonction deu paramètre `normalise`)
- contient la solution approchée obtenue avec `odeint`

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -.1, 11, -.1, 4

# choix de la condition initiale et du paramètre de normalisation
@interact(x_0=(0, xmax, .25), y_0=(0, ymax, .25), normalise=True)
# le dessin avec la solution pour la condition initiale (x_0,y_0)
def dyn_solution(x_0=0, y_0=0, normalise=True):


### 3) Observations

De combien d'orbite est composé la partie positive de l'ordonnée d'après les observations ? (décrivez votre expérience) 

**Solution :**


## Feuille de TD6, Exercice 1

### 1) La fonction de Cauchy

La fonction du problème de Cauchy autonome étudié dans cet exercice est :
$$
    F(Y) = (-y+xy, x+\frac{1}{2}(x^2-y^2)) \text{ pour } Y=(x,y).
$$

Définissez cette fonction.

In [ ]:
# La définition de F

# test
Y = np.array([[0, 1], [0, 1]])  # les arguments (0,0) et (1,1)
R = np.array([[0, 0], [0, 1]])
assert np.all(F(Y) == R), "F(0,0) = (0,0) and F(1,1) = (0,1)"

### 2) La rotation $\pm 120°$

Nous avons vu en TD que ce problème est invariant par la rotation de 120°. Pour pouvoir vérifier cette affirmation nous allons d'abord définir l'application `R120` qui applique la rotation à 120° qui opère sur des vecteurs `Y`.

In [ ]:
# La définition de la rotation R120 qui prend un 2-vecteur comme paramètre

# Ce n'est pas une façon optimale de définir la rotation à -120° = +240°.
# Donc vous pouvez faire mieux, si vous voulez.
R240 = lambda Y: R120(R120(Y))

# test
s = np.sqrt(3) / 2
Y = np.array([[0, 1], [0, 0]])
assert f"{R120(Y)[1,1]:.10f}" == "0.8660254038", f"{R120(Y)[1,1]:.10f} = -0.8660254038 ?"

### 2) Le graphique « dynamique » avec les solutions approchées

Inspirer vous des exemples traités pour afficher un graphique dynamique qui :
- permet de choisir la condition initiale `x_0, y_0` entre -10 et 10 ;
- affiche les isoclines $I_\infty$, $I_{\pm\frac{1}{\sqrt{3}}}$;
- qui affiche le champ de vecteurs normalisé
- contient la solution approchée obtenue avec `odeint`
- superpose la solution obtenue par rotation de 120° (dont la condition initiale est choisi de façon appropriée).

Le but de ce graphique est de faire la « preuve » visuelle de l'invariance par rotation.

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -10, 10, -10, 10

# choix de la condition initiale grâce à des curseurs
@interact(x_0=(xmin, xmax, .25), y_0=(ymin, ymax, .25))
# le dessin avec la solution pour la condition initiale (x_0,y_0)
def dyn_solution(x_0=1, y_0=np.sqrt(3)):


### 3) Observations

1. Combien de points fixes observez-vous ?
1. Avez-vous pu identifier numériquement les coordonnées de ces points fixes ?
1. Combien d'orbites sont contenus dans l'isocline $I_\infty$ d'après les observations ? (décrivez votre expérience)


**Solution :**


## Feuille de TD6, Exercice 2

La fonction dans cet exercice dépend du paramètre $\varepsilon$ :
$$ 
    F_\varepsilon(Y) = (y, -x+\varepsilon(x^2-1)y) \text{ pour } Y=(x,y).
$$

Vous avez la carte blanche pour réaliser l'étude de cet EDO. 